In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import text
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [38]:
train = pd.read_csv("/Users/ankitbaliyan/Documents/VS_Code/Ongoing projects/NLP_Disaster/dataset/train.csv")
test = pd.read_csv("/Users/ankitbaliyan/Documents/VS_Code/Ongoing projects/NLP_Disaster/dataset/test.csv")

train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [39]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [40]:
text_data=train['text'].astype('str')
text_data= text_data.apply(lambda x: x.lower())
text_data.head()

0    our deeds are the reason of this #earthquake m...
1               forest fire near la ronge sask. canada
2    all residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    just got sent this photo from ruby #alaska as ...
Name: text, dtype: object

In [41]:
# function to remove special characters
text_data= text_data.apply(lambda x: re.sub("[^0-9a-zA-Z]",' ', x))
text_data.head()

0    our deeds are the reason of this  earthquake m...
1               forest fire near la ronge sask  canada
2    all residents asked to  shelter in place  are ...
3    13 000 people receive  wildfires evacuation or...
4    just got sent this photo from ruby  alaska as ...
Name: text, dtype: object

In [42]:
# removing stopwords
#nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
stop_words

def remove_sw(review):
    tokens = word_tokenize(review)
    tokens = [w for w in tokens if not w in stop_words]
    return " ".join(tokens)
text_data = text_data.apply(remove_sw)
text_data.head()

0         deeds reason earthquake may allah forgive us
1                forest fire near la ronge sask canada
2    residents asked shelter place notified officer...
3    13 000 people receive wildfires evacuation ord...
4    got sent photo ruby alaska smoke wildfires pou...
Name: text, dtype: object

In [43]:
# Stemming - Not required as text is for product description

ps = PorterStemmer()

def stem(text):
    return ' '.join([ps.stem(word) for word in text.split()])

text_data= text_data.apply(stem)
text_data.head()


0            deed reason earthquak may allah forgiv us
1                 forest fire near la rong sask canada
2    resid ask shelter place notifi offic evacu she...
3    13 000 peopl receiv wildfir evacu order califo...
4    got sent photo rubi alaska smoke wildfir pour ...
Name: text, dtype: object

In [44]:
# initialise TF-IDF
#text_data=text_data.tolist()
#text_data.head()


In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 2))
matrix_tfidf = tf.fit_transform(text_data)

In [46]:
matrix_tfidf.shape

(7613, 72528)

In [47]:
train['text_treated']=text_data
train.head()

,id,keyword,location,text,target,text_treated
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may allah forgiv us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resid ask shelter place notifi offic evacu she...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13 000 peopl receiv wildfir evacu order califo...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...


In [48]:
X=train['text_treated']
Y=train['target']

In [49]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.25)
X_train.head()

7169    kasiakosek drive suck case know desk look like...
306     offici vid doublecup gt gt http co lfkmtzaekk ...
1817    brianroemmel ux fail emv peopl want insert rem...
1377    attack women health attack america health hill...
3332    trafford centr film fan angri odeon cinema eva...
Name: text_treated, dtype: object

In [50]:

#X_train_tf=text.TfidfVectorizer(input=X_train.tolist(),ngram_range=(1,1))
#X_train_tf= X_train_tf.fit_transform(X_train.tolist())

#X_val_tf=text.TfidfVectorizer(input=X_val.tolist(),ngram_range=(1,1))
#X_val_tf= X_val_tf.fit_transform(X_val.tolist())

In [51]:
tf=text.TfidfVectorizer()
X_train_tf=tf.fit_transform(X_train.tolist())

#X_val_tf=text.TfidfVectorizer()
X_val_tf= tf.transform(X_val.tolist())

In [52]:
X_train_tf.shape

(5709, 15024)

In [53]:
X_val_tf.shape

(1904, 15024)

In [54]:
y_val.shape

(1904,)

In [55]:
y_train.shape

(5709,)

In [56]:
mnb = MultinomialNB()
mnb.fit(X_train_tf, y_train)

MultinomialNB()

In [57]:
mnb.score(X_val_tf, y_val)

0.805672268907563

In [58]:
# Prediction on test set

test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [59]:
test_id=test['id']

In [60]:
test_text=test['text'].astype('str')
test_text= test_text.apply(lambda x: x.lower())
test_text.head()

0                   just happened a terrible car crash
1    heard about #earthquake is different cities, s...
2    there is a forest fire at spot pond, geese are...
3             apocalypse lighting. #spokane #wildfires
4        typhoon soudelor kills 28 in china and taiwan
Name: text, dtype: object

In [61]:
# function to remove special characters
test_text= test_text.apply(lambda x: re.sub("[^0-9a-zA-Z]",' ', x))
test_text.head()

0                   just happened a terrible car crash
1    heard about  earthquake is different cities  s...
2    there is a forest fire at spot pond  geese are...
3             apocalypse lighting   spokane  wildfires
4        typhoon soudelor kills 28 in china and taiwan
Name: text, dtype: object

In [62]:
test_text = test_text.apply(remove_sw)
test_text.head()

0                          happened terrible car crash
1    heard earthquake different cities stay safe ev...
2    forest fire spot pond geese fleeing across str...
3                apocalypse lighting spokane wildfires
4               typhoon soudelor kills 28 china taiwan
Name: text, dtype: object

In [63]:
test_text= test_text.apply(stem)
test_text.head()

0                             happen terribl car crash
1        heard earthquak differ citi stay safe everyon
2    forest fire spot pond gees flee across street ...
3                       apocalyps light spokan wildfir
4                typhoon soudelor kill 28 china taiwan
Name: text, dtype: object

In [64]:
test_text=test_text.tolist()

test_matrix_tfidf=tf.transform(test_text)

In [65]:
predictions=mnb.predict(test_matrix_tfidf)

In [66]:
predictions = pd.Series(predictions)
ids = pd.Series(test_id)

pred_df = pd.concat([ids, predictions], keys = ['id', 'target'], axis = 1)

#pred_df.to_csv('submission.csv',index = False)

In [67]:
pred_df.target.value_counts()

0    2206
1    1057
Name: target, dtype: int64